# The Amadeus API Test Flow

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

from google.cloud import secretmanager
client = secretmanager.SecretManagerServiceClient()

from google.cloud import secretmanager

client = secretmanager.SecretManagerServiceClient()

def get_secret(secret_id, project_id):
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

PROJECT_ID = "noetl-demo-19700101"

OPENAI_API_KEY = get_secret("openai-api-key", PROJECT_ID)
AMADEUS_API_KEY = get_secret("api-key-test-api-amadeus-com", PROJECT_ID)
AMADEUS_API_SECRET = get_secret("api-secret-test-api-amadeus-com", PROJECT_ID)

### Setting Up HTTP Requests

In [2]:
import requests
import json
from datetime import datetime
import requests
from requests.exceptions import RequestException
from urllib.parse import urlencode

AMADEUS_AUTH_URL = 'https://test.api.amadeus.com/v1/security/oauth2/token'
AMADEUS_BASE_URL = 'https://test.api.amadeus.com'
OPENAI_API_URL = 'https://api.openai.com/v1/chat/completions'

def get_amadeus_token(client_id, client_secret):
    try:
        response = requests.post(
            AMADEUS_AUTH_URL,
            data={
                'grant_type': 'client_credentials',
                'client_id': client_id,
                'client_secret': client_secret
            },
            headers={
                'Content-Type': 'application/x-www-form-urlencoded'
            }
        )
        response.raise_for_status()
        return response.json()['access_token']
    except RequestException as e:
        print(f"Error getting Amadeus token: {e}")
        return None

amadeus_token = get_amadeus_token(AMADEUS_API_KEY, AMADEUS_API_SECRET)
print(f"Amadeus API Token obtained: {'Success' if amadeus_token else 'Failed'}")

current_year = datetime.now().year

natlang_to_amadeus_system_prompt = f"""
You are a translator that converts natural-language travel queries into Amadeus API endpoints and parameters.

When given a request, follow these steps:
1. Understand what the user wants: flight search, hotel offers, airport search, etc.
2. Map the details (origin, destination, dates, passengers, etc.) to the correct REST API endpoint and parameters.
3. Return ONLY a JSON object with two properties:
   - "endpoint": The Amadeus API endpoint (e.g., "/v2/shopping/flight-offers")
   - "params": An object containing all query parameters

4. Use real values from user input, and apply correct formatting (e.g., ISO dates).
5. If values are ambiguous or missing, make reasonable assumptions.
6. If the user provides a travel date **without a year**, assume they mean {current_year}.
7. Always **limit the number of returned results to 3** where applicable.

Examples:

User:
> I want a one-way flight from SFO to JFK on September 15, 2025 for 1 adult.

A:
{{
  "endpoint": "/v2/shopping/flight-offers",
  "params": {{
    "originLocationCode": "SFO",
    "destinationLocationCode": "JFK",
    "departureDate": "2025-09-15",
    "adults": 1,
    "max": 3
  }}
}}

User:
> I'm looking for hotels in Paris from July 20 to July 25, up to 3 stars, no more than 200 euros per night.

A:
{{
  "endpoint": "/v2/shopping/hotel-offers",
  "params": {{
    "cityCode": "PAR",
    "checkInDate": "2025-07-20",
    "checkOutDate": "2025-07-25",
    "ratings": "3",
    "priceRange": "0-200",
    "max": 3
  }}
}}
"""


amadeus_to_natlang_system_prompt = """You are a helpful assistant that reads raw JSON responses from the Amadeus API and summarizes them into clear, human-readable language.

Your task is to:
1. Read and understand the JSON response structure (e.g. flights, hotels, etc.).
2. Identify key information relevant to the user (e.g. origin/destination, dates, price, airline, stops, hotel name, stars, location, price).
3. Summarize the most important details in **clean, natural English**.
4. Use lists, bullet points, and grouping if appropriate.
5. Do not show any raw JSON keys or code — just understandable sentences.
6. For IATA codes always write a transcription (i.e. SVO will be Sheremetyevo International Airport)

---

### Example 1: Flight Offers

**Input:**
```json
{
  "data": [
    {
      "id": "1",
      "price": { "total": "524.36", "currency": "USD" },
      "itineraries": [
        {
          "segments": [
            {
              "departure": {
                "iataCode": "SFO",
                "at": "2025-09-15T08:30:00"
              },
              "arrival": {
                "iataCode": "JFK",
                "at": "2025-09-15T17:05:00"
              },
              "carrierCode": "DL",
              "duration": "PT5H35M",
              "number": "1123"
            }
          ]
        }
      ],
      "travelerPricings": [
        { "travelerType": "ADULT", "fareDetailsBySegment": [{ "cabin": "ECONOMY" }] }
      ]
    }
  ]
}
Output:

Delta Air Lines flight DL1123 from SFO (San Francisco International Airport) to JFK (John F. Kennedy International Airport)

Departs at 08:30, arrives at 17:05 (duration: 5h 35m)

Economy class

Price: 524.36 USD for 1 adult"""


Amadeus API Token obtained: Success


### Translating Natural Language to Amadeus Query

**To try it out use this test prompt:** I would like to travel from LA to NY on New Year's Eve

In [3]:
def natlang_to_amadeus(natural_language_query):
    """Convert natural language to Amadeus API endpoint and parameters using OpenAI REST API"""
    try:
        response = requests.post(
            OPENAI_API_URL,
            headers={
                'Authorization': f'Bearer {OPENAI_API_KEY}',
                'Content-Type': 'application/json'
            },
            json={
                'model': 'gpt-4o',
                'messages': [
                    {
                        'role': 'system',
                        'content': natlang_to_amadeus_system_prompt
                    },
                    {
                        'role': 'user',
                        'content': natural_language_query
                    }
                ],
                'temperature': 0.1
            }
        )
        
        response.raise_for_status()
        result = response.json()
        content = result['choices'][0]['message']['content'].strip()

        try:
            amadeus_query = json.loads(content)
            return amadeus_query
        except json.JSONDecodeError:
            print(f"Error parsing OpenAI response as JSON: {content}")
            return None

    except RequestException as e:
        print(f"Error calling OpenAI API: {e}")
        return None

user_input = input()

amadeus_query = natlang_to_amadeus(user_input)
print(f"Amadeus Query: {json.dumps(amadeus_query, indent=2)}" if amadeus_query else "Failed to generate query")


 I would like to travel from LA to NY on New Year's Eve


Amadeus Query: {
  "endpoint": "/v2/shopping/flight-offers",
  "params": {
    "originLocationCode": "LAX",
    "destinationLocationCode": "JFK",
    "departureDate": "2025-12-31",
    "adults": 1,
    "max": 3
  }
}


### Executing Amadeus Query

In [4]:
def execute_amadeus_query(amadeus_query):
    """Execute Amadeus API query using REST API"""
    if not amadeus_query or not amadeus_token:
        print("Missing query or token")
        return None

    endpoint = amadeus_query.get('endpoint')
    params = amadeus_query.get('params', {})

    if not endpoint:
        print("No endpoint specified in query")
        return None

    try:
        url = f"{AMADEUS_BASE_URL}{endpoint}"
        headers = {
            'Authorization': f'Bearer {amadeus_token}',
            'Content-Type': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()

        result = response.json()
        return json.dumps(result, indent=2)

    except RequestException as e:
        print(f"Amadeus API Error: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"Error details: {json.dumps(error_detail, indent=2)}")
            except:
                print(f"Response content: {e.response.text}")
        return None
    except Exception as e:
        print(f"Execution Error: {e}")
        return None

amadeus_response = execute_amadeus_query(amadeus_query)
print(f"Amadeus Response: {amadeus_response[:500]}..." if amadeus_response and len(amadeus_response) > 500 else amadeus_response)


Amadeus Response: {
  "meta": {
    "count": 3,
    "links": {
      "self": "https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=LAX&destinationLocationCode=JFK&departureDate=2025-12-31&adults=1&max=3"
    }
  },
  "data": [
    {
      "type": "flight-offer",
      "id": "1",
      "source": "GDS",
      "instantTicketingRequired": false,
      "nonHomogeneous": false,
      "oneWay": false,
      "isUpsellOffer": false,
      "lastTicketingDate": "2025-07-27",
      "lastTicketingDateTime...


### Translating Amadeus Response to Natural Language

In [5]:
def amadeus_to_natlang(amadeus_json):
    """Convert Amadeus JSON response to natural language using OpenAI REST API"""
    if not amadeus_json:
        return "No response data to translate"

    try:
        response = requests.post(
            OPENAI_API_URL,
            headers={
                'Authorization': f'Bearer {OPENAI_API_KEY}',
                'Content-Type': 'application/json'
            },
            json={
                'model': 'gpt-4o',
                'messages': [
                    {
                        'role': 'system',
                        'content': amadeus_to_natlang_system_prompt
                    },
                    {
                        'role': 'user',
                        'content': amadeus_json
                    }
                ],
                'temperature': 0.3
            }
        )
        response.raise_for_status()

        result = response.json()
        return result['choices'][0]['message']['content'].strip()

    except RequestException as e:
        print(f"Error calling OpenAI API for translation: {e}")
        return "Failed to translate response to natural language"

total_result = amadeus_to_natlang(amadeus_response)
print("Final Result:")
print(total_result)


Final Result:
Here are three flight options from Los Angeles International Airport (LAX) to John F. Kennedy International Airport (JFK) on December 31, 2025, offered by JetBlue Airways:

### Option 1:
- **Flight Details:**
  - **First Segment:** 
    - Flight B6 492
    - Departs LAX at 21:45 from Terminal 5
    - Arrives at Buffalo Niagara International Airport (BUF) at 05:25 on January 1, 2026
    - Duration: 4 hours 40 minutes
    - Aircraft: Airbus A320
  - **Second Segment:**
    - Flight B6 2001
    - Departs BUF at 07:45
    - Arrives at JFK at 09:12 at Terminal 5
    - Duration: 1 hour 27 minutes
    - Aircraft: Airbus A220-300
- **Total Duration:** 8 hours 27 minutes
- **Class:** Economy (Blue Basic)
- **Price:** 182.82 EUR (approximately includes additional services like checked bags for 34.49 EUR)
- **Amenities:** Includes 2 cabin bags, snacks, and non-alcoholic drinks. Checked bags, extra legroom, and alcoholic drinks are chargeable.

### Option 2:
- **Flight Details:**
  -